In [1]:
from pathlib import Path
import pandas as pd

Load the orders data which was created in the [transform stage](./tickets.ipynb).

In [2]:
def prepare(data: pd.DataFrame) -> pd.DataFrame:
    data = data.loc[
        (
            data.public_event == True
        ) & (
            ~data.project_name.isin([
                'Skills Development',
            ])
        )
    ]
    data = data.drop(columns=['public_event'])
    return data


orders = pd.read_csv(
    '../../../../data/metrics/events/tickets/orders.csv',
    usecols=[
        'created_at',
        'project_name',
        'public_event',
        'number_of_tickets'
    ],
    parse_dates=['created_at']
).pipe(prepare)

Define functions to count and sum tickets, and to append cumulative versions of those.

In [3]:
def count_and_sum(series):
    return pd.concat([
        series.count().to_frame('orders'),
        series.sum().to_frame('tickets'),
    ], axis=1)


def cumulate(df: pd.DataFrame) -> pd.DataFrame:
    return pd.concat([
        df,
        df.cumsum().rename(columns=lambda x: f'cumulative_{x}')
    ], axis=1)

Create output directory

In [4]:
OUTPUT_DIR = Path('../../../../docs/metrics/tickets/_data/')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

Count / sum the orders and tickets by project code and save to a JSON file

In [5]:
by_project = orders.groupby('project_name').number_of_tickets.pipe(count_and_sum)
by_project.loc['total']= by_project.sum()
by_project.T.to_json(
  OUTPUT_DIR.joinpath('headlines.json'),
  indent=2
)

Add `day` column

In [6]:
orders['day'] = orders.created_at.dt.floor('D')

In [7]:
orders.loc[:, ['created_at', 'day']].head()

,created_at,day
176,2023-03-30 16:58:03,2023-03-30
177,2023-03-30 18:44:08,2023-03-30
180,2023-03-30 20:32:43,2023-03-30
181,2023-03-30 21:27:59,2023-03-30
183,2023-03-30 22:56:31,2023-03-30


Save table cumulative weekly per project

In [8]:
orders.groupby(
  ['day', 'project_name']
).number_of_tickets.pipe(
  count_and_sum
).tickets.unstack().fillna(0).resample('W-FRI').sum().cumsum().rename_axis('week').to_csv(
  OUTPUT_DIR.joinpath('by_project_cumulative_weekly.csv')
)

Save cumulative monthly sales

In [9]:
orders.groupby(['day']).number_of_tickets.pipe(count_and_sum).resample('MS').sum().pipe(cumulate).rename_axis('month').to_csv(
  OUTPUT_DIR.joinpath('monthly_cumulative.csv')
)